## **Analisis y clasificación de las reviews y comentarios en el sector de los videojuegos**
## **WEB SCRAPPING**

Eugenio Alberto Moreda 

**Importamos librerías**

In [ ]:
import requests
from bs4 import BeautifulSoup
import nltk
import pandas as pd
import time
from time import sleep, strftime
import random
from random import randint
import shutil
import pickle
from selenium import webdriver
import os
import numpy as np
import re
import matplotlib.pyplot as plt

**Web Scrapping**

Creación del dataframe

In [ ]:
df_usuarios = pd.DataFrame(columns = [
'Titulo', 'Plataforma', 'Fecha_salida', 
            'Puntuacion_usuarios',
            'Numero_criticas_usuarios_positivas',
            'Numero_criticas_usuarios_mixtas' ,
            'Numero_criticas_usuarios_negativas',
            'Puntuaciones_usuarios'  ,
            'Usuarios' ,
            'Fecha_review_usuarios',
            'Reviews_usarios' ,
            'Usuarios_ayudados' ,
            'Usuarios_ayudados_total' 
    ])

Iniciar el navegador y obtener los enlaces que necesitamos para hacer el web scrapping. En este caso se muestran para obtneer los enlaces de los comentarios de usuarios en ps4, pero se podría ajustar a los que se buscase.

In [ ]:
chrome_driver = r"C:\Users\Eugenio\Documents\Bootcamp data science\chromedriver"
navegador = webdriver.Chrome(executable_path = chrome_driver)
user_urls=[]
for i in range (20):
    navegador.get('https://www.metacritic.com/browse/games/score/metascore/all/ps4/filtered?page='+str(i))
    source = navegador.page_source
    soup = BeautifulSoup(source)
    for item in soup.find_all('div', class_='clamp-userscore'):
        for link in item.find_all('a'):
            user_urls.append(link.get('href'))
    sleep(3)

Variables recordatorio

In [1]:
videojuego_user=0
lista_repetir_final=0
errores_comentario=[]
primera=0

Inicio navegador

In [ ]:
chrome_driver = r"C:\Users\Eugenio\Documents\Bootcamp data science\chromedriver"
navegador = webdriver.Chrome(executable_path = chrome_driver)

navegador.get('https://www.metacritic.com') 
sleep(2)
navegador.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()

Es este caso se muestra para extraer los comentarios de usuarios, pero podría hacer igualmente para las reseñas de medios cambiando los enlaces que se le proporciona.

In [ ]:
while True:
    try:
        try:
            navegador.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        except:
            pass 
        for p, url_user in enumerate (user_urls[lista_repetir[videojuego_user]:], videojuego_user):
            videojuego_user= p
            
            navegador.get('https://www.metacritic.com'+str(url_user))
            sleep(randint(3,5))
            source = navegador.page_source
            soup = BeautifulSoup(source)     
            
            user_scores=[]
            users=[]
            date_user_score=[]
            user_reviews=[]
            ayudados=[]
            total_ayudados=[]
            
            titulo=soup.find('div', class_='product_title').text.strip().split('\n')[0].strip()
            plataforma=soup.find('div', class_='product_title').text.strip().split('\n')[-1].strip()
            try:
                fecha_salida=soup.find('li', class_='summary_detail release_data').find('span', class_="data").text
            except:
                fecha_salida= np.nan
            try:
                user_score=soup.find('div', class_="metascore_w user large game positive").text
            except:
                try:
                    user_score=soup.find('div', class_="metascore_w user large game mixed").text
                except:
                    try:
                        user_score=soup.find('div', class_="metascore_w user large game negative").text
                    except:
                        user_score=np.nan
            try:    
                numer_user_score_positive = soup.find('div', class_='score_distribution').find_all('span', class_='count')[0].text
                numer_user_score_mixed = soup.find('div', class_='score_distribution').find_all('span', class_='count')[1].text
                numer_user_score_negative = soup.find('div', class_='score_distribution').find_all('span', class_='count')[2].text
            except:
                numer_user_score_positive = np.nan
                numer_user_score_mixed = np.nan
                numer_user_score_negative = np.nan
                                
            try:
                ultima_pagina=int(soup.find('li', class_='page last_page').find('a', class_='page_num').text)
            except:
                try:
                    ultima_pagina=int(soup.find('li', class_='page last_page active_page').find('a', class_='page_num').text)
                except:
                    ultima_pagina=1
                    
            for pri, paginas in enumerate (range (primera,ultima_pagina), primera):
                primera=pri
                navegador.get('https://www.metacritic.com'+str(url_user)+'?page='+str(paginas))
                sleep(randint(3,5))
                source = navegador.page_source
                soup = BeautifulSoup(source)
                try:
                    soup.find('div', class_='msg msg_no_reviews')
                    user_scores=np.nan
                    users=np.nan
                    date_user_score=np.nan
                    user_reviews=np.nan
                    ayudados=np.nan
                    total_ayudados=np.nan 
                except:
                    numero_criticas_usuarios_actual = len(soup.find('ol', class_="reviews user_reviews").find_all('div', class_="review_grade"))
                    if numero_criticas_usuarios_actual== 1:
                        user_scores.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review last_review').find('div', class_='review_grade').text.split('\n')[1])
                        users.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review last_review').find('div', class_='name').text.split('\n')[1])
                        date_user_score.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review last_review').find('div', class_='date').text)
                        try:
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review last_review').find('div', class_='review_body').find('span', class_='blurb blurb_expanded').text)                        
                        except:
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review last_review').find('div', class_='review_body').text.split('\n')[1].strip())
                        ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review last_review').find('span', class_='total_ups').text)
                        total_ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review last_review').find('span', class_='total_thumbs').text)
                    elif numero_criticas_usuarios_actual== 2:
                        user_scores.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='review_grade').text.split('\n')[1])
                        users.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='name').text.split('\n')[1])
                        date_user_score.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='date').text)
                        try:
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='review_body').find('span', class_='blurb blurb_expanded').text)
                        except:
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='review_body').text.split('\n')[1].strip())
                        ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('span', class_='total_ups').text)
                        total_ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('span', class_='total_thumbs').text) 
                        user_scores.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='review_grade').text.split('\n')[1])
                        users.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='name').text.split('\n')[1])
                        date_user_score.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='date').text)
                        try:
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='review_body').find('span', class_='blurb blurb_expanded').text)                            
                        except: 
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='review_body').text.split('\n')[1].strip())
                        ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('span', class_='total_ups').text)
                        total_ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('span', class_='total_thumbs').text)       
                    else:
                        user_scores.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='review_grade').text.split('\n')[1])
                        users.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='name').text.split('\n')[1])
                        date_user_score.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='date').text)
                        try:
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='review_body').find('span', class_='blurb blurb_expanded').text)
                        except:
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('div', class_='review_body').text.split('\n')[1].strip())
                        ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('span', class_='total_ups').text)
                        total_ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review first_review').find('span', class_='total_thumbs').text)
                        try:  
                            for i in range (len(soup.find('ol', class_='reviews user_reviews').find_all('li', class_='review user_review'))):
                                user_scores.append(soup.find('ol', class_='reviews user_reviews').find_all('li', class_='review user_review')[i].find('div', class_='review_grade').text.split('\n')[1])
                                users.append(soup.find('ol', class_='reviews user_reviews').find_all('li', class_='review user_review')[i].find('div', class_='name').text.split('\n')[1])
                                date_user_score.append(soup.find('ol', class_='reviews user_reviews').find_all('li', class_='review user_review')[i].find('div', class_='date').text)
                                try:
                                     user_reviews.append(soup.find('ol', class_='reviews user_reviews').find_all('li', class_='review user_review')[i].find('div', class_='review_body').find('span', class_='blurb blurb_expanded').text)
                                except:
                                    user_reviews.append(soup.find('ol', class_='reviews user_reviews').find_all('li', class_='review user_review')[i].find('div', class_='review_body').text.split('\n')[1].strip())
                                ayudados.append(soup.find('ol', class_='reviews user_reviews').find_all('li', class_='review user_review')[i].find('span', class_='total_ups').text)
                                total_ayudados.append(soup.find('ol', class_='reviews user_reviews').find_all('li', class_='review user_review')[i].find('span', class_='total_thumbs').text)
                        except:
                            pass                    
                        user_scores.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='review_grade').text.split('\n')[1])
                        users.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='name').text.split('\n')[1])
                        date_user_score.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='date').text)
                        try:
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='review_body').find('span', class_='blurb blurb_expanded').text)                            
                        except: 
                            user_reviews.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('div', class_='review_body').text.split('\n')[1].strip())
                        ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('span', class_='total_ups').text)
                        total_ayudados.append(soup.find('ol', class_='reviews user_reviews').find('li', class_='review user_review last_review').find('span', class_='total_thumbs').text)                                       
                sleep(randint(3,5))                              
            
            new_row = {
                    'Titulo' : titulo,
                    'Plataforma' : plataforma,
                    'Fecha_salida' : fecha_salida,
                    'Puntuacion_usuarios' : user_score,
                    'Numero_criticas_usuarios_positivas' :  numer_user_score_positive,
                    'Numero_criticas_usuarios_mixtas'    :  numer_user_score_mixed,
                    'Numero_criticas_usuarios_negativas' :  numer_user_score_negative,
                    'Puntuaciones_usuarios'   :  user_scores,
                    'Usuarios'                :  users,
                    'Fecha_review_usuarios'   :  date_user_score,
                    'Reviews_usarios'         :  user_reviews,
                    'Usuarios_ayudados'       :  ayudados,
                    'Usuarios_ayudados_total' : total_ayudados     
                 }
            df_usuarios = df_usuarios.append(new_row, ignore_index=True)
            primera=0
        sleep(randint(4,7))
        break
    except:
        navegador.refresh()
        errores_comentario.append(titulo)
        continue   
    break
df_usuarios.to_pickle('/Users/Eugenio/Documents/comentarios_presentacion.pkl')